<a href="https://colab.research.google.com/github/Terence0408/Teach_code/blob/master/Deep_Learning_Python_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


# Load package

Tutorial: https://www.tensorflow.org/tutorials/keras


In [18]:
import tensorflow as tf
from tensorflow import keras

from gensim.models import Word2Vec
import gensim.downloader as api

from keras.preprocessing import sequence
from keras.datasets import imdb

from datetime import datetime

# Word Embeddings

## Train by yourself

In [19]:
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1, vector_size=20)

# summarize the loaded model
print(model)

Word2Vec<vocab=14, vector_size=20, alpha=0.025>


In [20]:
# access vector for one word
print(model.wv['sentence'])

[-0.00268114  0.00118216  0.02551675  0.04504637 -0.04651475 -0.03558404
  0.03229436  0.04486494 -0.02507714 -0.01881686  0.03690252 -0.00766736
 -0.02268307  0.03277026 -0.0243008  -0.00908009  0.0143829   0.00495937
 -0.04142607 -0.04724409]


In [21]:
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec<vocab=14, vector_size=20, alpha=0.025>


## Use trained

In [22]:
# load glove vectors
model = api.load("glove-twitter-25")

# show words that similar to word 'cat'
model.most_similar("king")

[('prince', 0.9337409734725952),
 ('queen', 0.9202421307563782),
 ('aka', 0.9176921844482422),
 ('lady', 0.9163240790367126),
 ('jack', 0.9147354364395142),
 ("'s", 0.9066898226737976),
 ('stone', 0.8982374668121338),
 ('mr.', 0.8919409513473511),
 ('the', 0.889343798160553),
 ('star', 0.8892088532447815)]

In [23]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('meets', 0.8841924071311951),
 ('prince', 0.832163393497467),
 ('queen', 0.8257461190223694),
 ('’s', 0.8174097537994385),
 ('crow', 0.813499391078949),
 ('hunter', 0.8131037950515747),
 ('father', 0.8115834593772888),
 ('soldier', 0.81113600730896),
 ('mercy', 0.8082392811775208),
 ('hero', 0.8082264065742493)]

# RNN
Recurrent Neural Networks

## Data

In [24]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)

# Loading data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

25000 train sequences
25000 test sequences


In [25]:
X_train = sequence.pad_sequences(X_train, maxlen = 80)
X_test = sequence.pad_sequences(X_test, maxlen = 80)

print('x_train shape:', X_train.shape)
X_train[0]

x_train shape: (25000, 80)


array([   15,   256,     4,     2,     7,  3766,     5,   723,    36,
          71,    43,   530,   476,    26,   400,   317,    46,     7,
           4, 12118,  1029,    13,   104,    88,     4,   381,    15,
         297,    98,    32,  2071,    56,    26,   141,     6,   194,
        7486,    18,     4,   226,    22,    21,   134,   476,    26,
         480,     5,   144,    30,  5535,    18,    51,    36,    28,
         224,    92,    25,   104,     4,   226,    65,    16,    38,
        1334,    88,    12,    16,   283,     5,    16,  4472,   113,
         103,    32,    15,    16,  5345,    19,   178,    32],
      dtype=int32)

In [26]:
y_train[0]

1

## Model

### RNN

In [27]:
# Build the Keras Sequential model
RNN_model = keras.models.Sequential()
RNN_model.add(keras.layers.Embedding(max_features, 128))
RNN_model.add(keras.layers.SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False))
RNN_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model
RNN_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

RNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2593025 (9.89 MB)
Trainable params: 2593025 (9.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Evalute the model
start_time = datetime.now() # Record the start time

RNN_model.fit(X_train, y_train,
              batch_size = 32,
              epochs=1,
          validation_data=(X_test, y_test))

# Record the run time
end_time = datetime.now()
RNN_time = end_time - start_time

RNN_score, RNN_acc = RNN_model.evaluate(X_test, y_test, batch_size = 32)
print('Test score:', RNN_score)
print('Test accuracy:', RNN_acc)
print('Duration: {}'.format(RNN_time))

782/782 [==============================] - 6s 8ms/step - loss: 0.4982 - accuracy: 0.7605
Test score: 0.49816885590553284
Test accuracy: 0.7605199813842773
Duration: 0:02:23.739882


### LSTM

In [29]:
# Build the Keras Sequential model
LSTM_model = keras.models.Sequential()
LSTM_model.add(keras.layers.Embedding(max_features, 128))
LSTM_model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False))
LSTM_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model
LSTM_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

LSTM_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2691713 (10.27 MB)
Trainable params: 2691713 (10.27 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Evalute the model
start_time = datetime.now() # Record the start time
LSTM_model.fit(X_train, y_train,
               batch_size = 32,
               epochs=1,
               validation_data=(X_test, y_test))

# Record the run time
end_time = datetime.now()
LSTM_time = end_time - start_time

LSTM_score, LSTM_acc = LSTM_model.evaluate(X_test, y_test, batch_size = 32)
print('Test score:', LSTM_score)
print('Test accuracy:', LSTM_acc)
print('Duration: {}'.format(LSTM_time))

782/782 [==============================] - 20s 26ms/step - loss: 0.4824 - accuracy: 0.8235
Test score: 0.48235321044921875
Test accuracy: 0.8235200047492981
Duration: 0:05:25.089539


### GRU

In [31]:
# Build the Keras Sequential model
GRU_model = keras.models.Sequential()
GRU_model.add(keras.layers.Embedding(max_features, 128))
GRU_model.add(keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False))
GRU_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model
GRU_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

GRU_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         2560000   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2659201 (10.14 MB)
Trainable params: 2659201 (10.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Evalute the model
start_time = datetime.now() # Record the start time
GRU_model.fit(X_train, y_train,
               batch_size = 32,
               epochs=1,
               validation_data=(X_test, y_test))

# Record the run time
end_time = datetime.now()
GRU_time = end_time - start_time

GRU_score, GRU_acc = GRU_model.evaluate(X_test, y_test, batch_size = 32)
print('Test score:', GRU_score)
print('Test accuracy:', GRU_acc)
print('Duration: {}'.format(GRU_time))

782/782 [==============================] - 20s 25ms/step - loss: 0.3454 - accuracy: 0.8498
Test score: 0.3454147279262543
Test accuracy: 0.8497599959373474
Duration: 0:05:24.805812


### biLSTM

In [33]:
# Build the Keras Sequential model
biLSTM_model = keras.models.Sequential()
biLSTM_model.add(keras.layers.Embedding(max_features, 128))
biLSTM_model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False)))
biLSTM_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model
biLSTM_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

biLSTM_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2823425 (10.77 MB)
Trainable params: 2823425 (10.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Evalute the model
start_time = datetime.now() # Record the start time
biLSTM_model.fit(X_train, y_train,
               batch_size = 32,
               epochs=1,
               validation_data=(X_test, y_test))

# Record the run time
end_time = datetime.now()
biLSTM_time = end_time - start_time

biLSTM_score, biLSTM_acc = biLSTM_model.evaluate(X_test, y_test, batch_size = 32)
print('Test score:', biLSTM_score)
print('Test accuracy:', biLSTM_acc)
print('Duration: {}'.format(biLSTM_time))

782/782 [==============================] - 40s 51ms/step - loss: 0.3606 - accuracy: 0.8420
Test score: 0.3606014847755432
Test accuracy: 0.8419600129127502
Duration: 0:10:27.922729


### biLSTM_custom

In [35]:
# Build the Keras Sequential model
biLSTM_custom_model = keras.models.Sequential()
biLSTM_custom_model.add(keras.layers.Embedding(max_features, 128))

forward_layer = keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False)
backward_layer = keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= False, go_backwards=True)

biLSTM_custom_model.add(keras.layers.Bidirectional(forward_layer, backward_layer=backward_layer))
biLSTM_custom_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model
biLSTM_custom_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

biLSTM_custom_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               263168    
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2823425 (10.77 MB)
Trainable params: 2823425 (10.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Evalute the model
start_time = datetime.now() # Record the start time
biLSTM_custom_model.fit(X_train, y_train,
               batch_size = 32,
               epochs=1,
               validation_data=(X_test, y_test))

# Record the run time
end_time = datetime.now()
biLSTM_custom_time = end_time - start_time

biLSTM_custom_score, biLSTM_custom_acc = biLSTM_custom_model.evaluate(X_test, y_test, batch_size = 32)
print('Test score:', biLSTM_custom_score)
print('Test accuracy:', biLSTM_custom_acc)
print('Duration: {}'.format(biLSTM_custom_time))

782/782 [==============================] - 38s 48ms/step - loss: 0.3577 - accuracy: 0.8407
Test score: 0.3577262759208679
Test accuracy: 0.8406800031661987
Duration: 0:10:28.919293


## Compare

In [37]:
print('RNN Test accuracy:', RNN_acc)
print('LSTM Test accuracy:', LSTM_acc)
print('GRU Test accuracy:', GRU_acc)
print('biLSTM Test accuracy:', biLSTM_acc)
print('biLSTM_custom Test accuracy:', biLSTM_custom_acc)

RNN Test accuracy: 0.7605199813842773
LSTM Test accuracy: 0.8235200047492981
GRU Test accuracy: 0.8497599959373474
biLSTM Test accuracy: 0.8419600129127502
biLSTM_custom Test accuracy: 0.8406800031661987


In [38]:
print('RNN Test accuracy:', RNN_time)
print('LSTM Test accuracy:', LSTM_time)
print('GRU Test accuracy:', GRU_time)
print('biLSTM Test accuracy:', biLSTM_time)
print('biLSTM_custom Test accuracy:', biLSTM_custom_time)

RNN Test accuracy: 0:02:23.739882
LSTM Test accuracy: 0:05:25.089539
GRU Test accuracy: 0:05:24.805812
biLSTM Test accuracy: 0:10:27.922729
biLSTM_custom Test accuracy: 0:10:28.919293
